In [12]:
import os
import torch
from openbabel import pybel

from utils.model_utils import load_model
from utils.pocket_utils import save_pocket_mol
from utils.data_utils import prepare_data, process_protein_data

In [13]:
model_path = "./ckpt/model0.pth"
model = load_model(model_path)

In [14]:
input_file = "./coach420"
output_path = "./output/coach420"
file_format = "mol2"

In [15]:
val_Dataloader = prepare_data(input_file, file_format)

results = []

for i, data in enumerate(val_Dataloader):
    input, origin, step, name, mol = data
    mol = next(pybel.readfile(file_format, mol[0]))
    input = input.cuda()
    with torch.no_grad():
        output = model(input)
    pockets, binding_score = save_pocket_mol(output.cpu(), origin[0], step[0], mol)

    for j, pocket in enumerate(pockets):
        folder_name = output_path + '/' + str(name).split("'")[1]
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
            
        pocket_filename = f"{folder_name}/pocket{j}_score_{binding_score:.4f}.{file_format}"
        pocket.write(file_format, pocket_filename, overwrite=True)